In [1]:
# 1. Install required packages
%pip install pandas ipython-sql prettytable
import prettytable
prettytable.DEFAULT = 'DEFAULT'

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 2. Import required libraries
import sqlite3, csv
%load_ext sql



In [3]:
# 3. Create database connection
con = sqlite3.connect("/Users/art/Public/chicago_socioeconomic/FinalDB.db")
cur = con.cursor()

In [4]:
# 4. Enable SQL magic
%sql sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db

In [5]:
# create 3 dataframes one for each dataset
#df1 census data
import pandas

df1 = pandas.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
df1.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")


78

In [6]:
#df 2  chigago public schools data
df2 = pandas.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
df2.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists='replace', index=False,method="multi")

566

In [7]:
# chicago crime data
df3 = pandas.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
df3.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False,method="multi")

533

In [8]:
# make sure the table got created
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


name
CENSUS_DATA
CHICAGO_PUBLIC_SCHOOLS
CHICAGO_CRIME_DATA


In [9]:
#Find the total number of crimes recorded in the CRIME table
%sql SELECT count (CASE_NUMBER) from CHICAGO_CRIME_DATA; #533

 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.
(sqlite3.OperationalError) near "#533": syntax error
[SQL: #533]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [10]:
#List community area names and numbers with per capita income less than 11000
%sql SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME<11000;

 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0


In [11]:
#List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)
%sql SELECT `CASE_NUMBER` FROM CHICAGO_CRIME_DATA WHERE `DESCRIPTION` LIKE '%minor%';

 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


In [12]:
#List all kidnapping crimes involving a child?
%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE='KIDNAPPING' AND DESCRIPTION LIKE '%child%'

 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


In [13]:
#List the kind of crimes that were recorded at schools.  (No repetitions)
%sql SELECT distinct (primary_type) FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION like 'SCHOOL%';

 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


In [14]:
#List the type of schools along with the average safety score for each type.
%sql SELECT `Elementary, Middle, or High School`, AVG(SAFETY_SCORE) from CHICAGO_PUBLIC_SCHOOLS group by 1;

 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


"Elementary, Middle, or High School",AVG(SAFETY_SCORE)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


In [15]:
#List 5 community areas with highest % of households below poverty line
%sql SELECT COMMUNITY_AREA_NUMBER, PERCENT_HOUSEHOLDS_BELOW_POVERTY from CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;

 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,56.5
37.0,51.2
68.0,46.6
29.0,43.1
27.0,42.4


In [16]:
#Which community area is most crime prone? Display the coumminty area number only.

%sql SELECT  c.COMMUNITY_AREA_NUMBER, count (cr.id) from  CENSUS_DATA c, CHICAGO_CRIME_DATA cr where cr.COMMUNITY_AREA_NUMBER = c.COMMUNITY_AREA_NUMBER group by 1 order by 2 desc;



 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,count (cr.id)
25.0,43
23.0,22
68.0,21
29.0,16
28.0,16
8.0,15
71.0,14
24.0,13
67.0,12
66.0,12


In [17]:
#Use a sub-query to find the name of the community area with highest hardship index
%sql select COMMUNITY_AREA_NAME from CENSUS_DATA where HARDSHIP_INDEX in (select max(HARDSHIP_INDEX) from CENSUS_DATA);



 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


In [18]:
#Use a sub-query to determine the Community Area Name with most number of crimes?

%sql select COMMUNITY_AREA_NAME from CENSUS_DATA where COMMUNITY_AREA_NUMBER = (select COMMUNITY_AREA_NUMBER from CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY count (*) DESC);

 * sqlite:////Users/art/Public/chicago_socioeconomic/FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
